In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install automl_alex.
#!pip install --quiet automl_alex

In [1]:
import automl_alex
import sklearn
import time
from automl_alex import AutoML, AutoMLClassifier
print(automl_alex.__version__)

0.07.04


In [2]:
RANDOM_SEED = 42

# Classifier

## Data

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
dataset = fetch_openml(name='credit-g', version=1, as_frame=True)
dataset.target = dataset.target.astype('category').cat.codes
dataset.data.head(5)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,22.0,none,own,1.0,skilled,1.0,none,yes
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,3.0,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,4.0,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,4.0,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes


In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, 
                                                    dataset.target,
                                                    test_size=0.25, 
                                                    random_state=RANDOM_SEED,)
X_train.shape, X_test.shape

((750, 20), (250, 20))

## Model

In [5]:
model = AutoML(X_train, y_train, X_test, type_of_estimator='classifier', random_state=RANDOM_SEED)

# or Simply
model = AutoMLClassifier(X_train, y_train, X_test, random_state=RANDOM_SEED)

In [6]:
%%time
predict_test, predict_train = model.fit_predict(timeout=1000, verbose=2)


 Opt StackingModels
One iteration takes ~ 0.3 sec
Start Auto calibration parameters
Start optimization with the parameters:
CV_Folds =10
Score_CV_Folds =3
Opt_lvl =3
Cold_start =159.0
Early_stoping =318.0
Metric =roc_auc_score
Direction =maximize
Default model roc_auc_score = 0.649
########################################
Optimize: : 299it [11:43,  2.35s/it, CatBoost Best Score roc_auc_score = 0.8137 +- 0.043145]

 Predict from StackingModels
  0%|          | 0/10 [00:00<?, ?it/s]
 Mean Score roc_auc_score on 30 Folds: 0.7959 std: 0.050629
 10%|█         | 1/10 [00:27<04:10, 27.87s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7881 std: 0.046341
 20%|██        | 2/10 [01:17<04:36, 34.53s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7933 std: 0.050665
 30%|███       | 3/10 [02:15<04:50, 41.43s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7914 std: 0.048463
 40%|████      | 4/10 [03:08<04:29, 44.85s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7976 std: 0.053204
 50%|█████     | 5/10 [03

In [7]:
predict_test[:5]

array([0.41433388, 0.47800449, 0.54230689, 0.39425326, 0.24914714])

In [8]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predict_test),4))

Test AUC:0.791


In [12]:
model.stack_models_cfgs

,score_opt,model_score,score_std,model_name,model_param,wrapper_params,cat_encoder,cv_folds
131,0.7663,0.8202,0.053903,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},HelmertEncoder,10
263,0.7706,0.8137,0.043145,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},HelmertEncoder,10
156,0.7645,0.8133,0.048850,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},HelmertEncoder,10
25,0.7585,0.8085,0.050013,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},HelmertEncoder,10
228,0.7396,0.8079,0.068280,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},HelmertEncoder,10
12,0.7527,0.8067,0.053999,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},HelmertEncoder,10
198,0.7540,0.8053,0.051257,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},HelmertEncoder,10
269,0.7452,0.8051,0.059851,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},HelmertEncoder,10
41,0.7392,0.8051,0.065928,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},HelmertEncoder,10
34,0.7363,0.8002,0.063944,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},HelmertEncoder,10
